In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [2]:
#np.random.seed(404)
np.set_printoptions(precision=2, suppress=True)

In [3]:
# Read data
data = pd.read_excel('Modelling_Data_Phase_2.xlsx', sheet_name ='Seed', index_col=False) #change the sheet name according to the tabs to run the various cycles in the current Phase
data.head()


,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,SHA,XCO
0,0.1010,0.478,0.213,0.208,260,2.0,24000,158.555963,0.144916,0.130594
1,0.1010,0.202,0.480,0.217,260,2.0,24000,220.116161,0.107552,0.237733
2,0.1170,0.421,0.104,0.360,260,2.0,24000,242.502579,0.157901,0.165786
3,0.0966,0.201,0.230,0.465,260,2.0,24000,270.214203,0.152576,0.195074
4,0.1280,0.112,0.401,0.359,260,2.0,24000,278.589571,0.164758,0.189697


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Zr      80 non-null     float64
 1   Cu      80 non-null     float64
 2   Co      80 non-null     float64
 3   Fe      80 non-null     float64
 4   T       80 non-null     float64
 5   H2:CO   80 non-null     float64
 6   GHSV    80 non-null     float64
 7   STY     80 non-null     float64
 8   SHA     80 non-null     float64
 9   XCO     80 non-null     float64
dtypes: float64(10)
memory usage: 6.4 KB


In [71]:
# General stastistical data
data.describe()

,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,SHA,XCO
count,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000
mean,0.142002,0.159923,0.191421,0.506612,267.095203,2.119577,34506.014120,374.464180,0.114882,0.348309
std,0.132488,0.143871,0.103284,0.200800,13.490404,0.493730,18301.333128,257.928242,0.024351,0.199520
min,0.048322,0.049333,0.049378,0.048463,240.000000,1.150000,10250.000000,13.552616,0.048056,0.000000
25%,0.093115,0.085033,0.116681,0.359000,260.000000,1.980000,24000.000000,160.296258,0.100104,0.185518
50%,0.101033,0.102313,0.192171,0.605046,260.000000,2.000000,24000.000000,340.658321,0.114196,0.361493
75%,0.128000,0.132793,0.214183,0.651355,279.202165,2.415000,38473.345588,532.698817,0.130456,0.454711
max,0.846036,0.663680,0.480000,0.806660,314.300000,3.150000,90000.000000,1098.461455,0.164758,0.940109


In [72]:
# Define X and Y from the data

X = data[['Zr ', 'Cu ', 'Co ', 'Fe','T','H2:CO','GHSV']]
Y = data['STY']
Y2 = pd.DataFrame({'YHA': data['SHA'] * data['XCO']})
Y3 = pd.concat([Y, Y2], axis=1)

In [73]:
#Check data for X or Y
Y

,STY,YHA
0,158.555963,0.018925
1,220.116161,0.025569
2,242.502579,0.026178
3,270.214203,0.029764
4,278.589571,0.031254
...,...,...
75,861.337713,0.051697
76,892.820722,0.051531
77,899.161813,0.051969
78,1088.568786,0.053519


In [75]:
# Feature normalization for X. 

from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler(feature_range=(0,1))
mm.fit(np.array(X)[:,4:])

def scalex(x):
    x = np.array(x)
    x[:,4:] = mm.transform(x[:,4:])
    return x

def scalex_const(x_const):
    x_const = np.array(x_const)
    x_const = mm.transform(x_const)
    return x_const

X = scalex(X)

In [76]:
# Feature standardization for Y, applied to Y. Invert Y by multiplying with -1 to form a MAXIMIZATION problem as BO is MINIMIZATION by default

from sklearn.preprocessing import StandardScaler

stc = StandardScaler()
stc.fit(np.array(Y))

def scaley(y):
    y_scaled = -1*stc.transform(np.array(y))
    return y_scaled

def scaley_inv(y_scaled):
    y = stc.inverse_transform(np.array(-1*y_scaled))
    return y

Y = scaley(Y)

In [77]:
#Check the values of X or Y
X

array([[0.101, 0.478, 0.213, 0.208, 0.269, 0.425, 0.172],
       [0.101, 0.202, 0.48 , 0.217, 0.269, 0.425, 0.172],
       [0.117, 0.421, 0.104, 0.36 , 0.269, 0.425, 0.172],
       [0.097, 0.201, 0.23 , 0.465, 0.269, 0.425, 0.172],
       [0.128, 0.112, 0.401, 0.359, 0.269, 0.425, 0.172],
       [0.128, 0.107, 0.161, 0.605, 0.269, 0.425, 0.172],
       [0.624, 0.052, 0.054, 0.269, 0.269, 0.425, 0.172],
       [0.051, 0.351, 0.357, 0.241, 0.269, 0.425, 0.172],
       [0.244, 0.266, 0.211, 0.279, 0.269, 0.425, 0.172],
       [0.279, 0.05 , 0.343, 0.328, 0.269, 0.425, 0.172],
       [0.146, 0.074, 0.183, 0.597, 0.269, 0.425, 0.172],
       [0.082, 0.099, 0.214, 0.605, 0.269, 0.425, 0.172],
       [0.846, 0.056, 0.05 , 0.048, 0.269, 0.425, 0.172],
       [0.499, 0.394, 0.055, 0.052, 0.269, 0.425, 0.172],
       [0.514, 0.051, 0.383, 0.052, 0.269, 0.425, 0.172],
       [0.108, 0.056, 0.129, 0.707, 0.269, 0.425, 0.172],
       [0.075, 0.057, 0.061, 0.807, 0.269, 0.425, 0.172],
       [0.094,

In [12]:
# Import necesarry libraries for Gaussian process regression 

from gpflow.models import GPR
from gpflow.models import SVGP
from gpflow.likelihoods import Gaussian
from gpflow.optimizers import Scipy
from gpflow.kernels import SquaredExponential as SE, Constant as C, White as W, SharedIndependent as SI
from gpflow.inducing_variables import SharedIndependentInducingVariables as SIIV, InducingPoints as IP
from sklearn.metrics import r2_score, mean_squared_error

In [78]:
#Define the kernels using squared exponential. The dimentions of lengthscales must match the number of input features
#This is a single objective task with 7 input features, where each feature correspond to the metal composition and reaction conditions

# single objective with 4 metals + reaction conditions
kernel = SE(lengthscales=[0.1,0.1,0.1,0.1,0.1,0.1,0.1])


# Gaussian process regression
gp_model = GPR((X, Y), kernel=kernel)

# Optimize the lengthscales
opt = Scipy()
opt.minimize(gp_model.training_loss, gp_model.trainable_variables)


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 47.86787923951954
        x: [ 1.381e+00  2.196e+03  1.982e+00 -6.563e-01 -3.852e-01
             2.135e+00  3.858e-01  2.090e+00 -2.944e+00]
      nit: 101
      jac: [ 2.521e-04 -1.222e-09 -3.436e-04 -9.863e-04  4.871e-04
             1.761e-04 -3.232e-04 -4.552e-05  1.043e-03]
     nfev: 115
     njev: 115
 hess_inv: <9x9 LbfgsInvHessProduct with dtype=float64>

In [79]:
from sklearn.metrics import r2_score, mean_squared_error

# After optimization, you can make predictions with the trained model
Y_pred_mean, _ = gp_model.predict_y(X)  # Predicted mean values

# Evaluate R^2 score
r2 = r2_score(Y, Y_pred_mean)

# Evaluate mean squared error (RMSE)
mse = mean_squared_error(Y, Y_pred_mean)
rmse = np.sqrt(mse)

# Print the evaluation metrics with limited decimal places

print("R^2 score: {:.2f}".format(r2))
print("Root Mean Squared Error: {:.2f}".format(rmse))

R^2 score: 0.96
Root Mean Squared Error: 0.20


In [80]:
# Check the optimized hyperparameters

optimized_lengthscales = gp_model.kernel.lengthscales.numpy()
print("Optimized Lengthscales:", optimized_lengthscales)

Optimized Lengthscales: [   1.605 2196.217    2.111    0.418    0.519    2.247    0.905]


# Bayesian optimization

In [16]:
from trieste.space import LinearConstraint
from trieste.space import Box

# Define lower and upper bounds for metal fractions and reaction conditions
Zr_lb = 0.1
Zr_ub = 0.1
Cu_lb = 0.05
Cu_ub = 0.2
Co_lb = 0.1
Co_ub = 0.3
Fe_lb = 0.4
Fe_ub = 0.8
T_lb = 250
T_ub = 300
H2_CO_lb = 1
H2_CO_ub = 4
GHSV_lb = 30000
GHSV_ub = 50000

const_lb = -10
const_ub = 10

const_mat = np.array([T_lb, H2_CO_lb, GHSV_lb, T_ub, H2_CO_ub, GHSV_ub]).reshape(2,3)
print(const_mat)
const_mat = scalex_const(const_mat)
print(const_mat)

T_lb = const_mat[0,0]
T_ub = const_mat[1,0]
H2_CO_lb = const_mat[0,1]
H2_CO_ub = const_mat[1,1]
GHSV_lb = const_mat[0,2]
GHSV_ub = const_mat[1,2]


# Define linear constraints. Apply lb and ub to the scalar product of the number vector and the feature vector

# Metal compositions + reaction conditions
constraints = [LinearConstraint(A=tf.constant
       ([[1, 1, 1, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], 
        [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]]), 
                                
        lb=tf.constant([1, Zr_lb, Cu_lb, Co_lb, Fe_lb, T_lb, H2_CO_lb, GHSV_lb]), 
        ub=tf.constant([1, Zr_ub, Cu_ub, Co_ub, Fe_ub, T_ub, H2_CO_ub, GHSV_ub]))]

constrained_search_space = Box([0, 0, 0, 0, const_lb, const_lb, const_lb], 
                               [1, 1, 1, 1, const_ub, const_ub, const_ub], 
                               constraints=constraints)

[[  250     1 30000]
 [  300     4 50000]]
[[ 0.25 -0.33  0.6 ]
 [ 1.5   1.67  1.27]]


In [17]:
# Essential functions for formatting data

from trieste.data import Dataset

def observer(in_):
    in_ = tf.convert_to_tensor(in_)
    out_, _ = gp_model.predict_y(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

def initial_data(in_, out_):
    in_ = tf.convert_to_tensor(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

In [18]:
# Import necessary libraries to build model

from trieste.models.gpflow import GaussianProcessRegression
from trieste.bayesian_optimizer import BayesianOptimizer
from trieste.acquisition.rule import EfficientGlobalOptimization
from trieste.acquisition.function import Fantasizer
from trieste.acquisition import LocalPenalization
from trieste.acquisition.function import ExpectedHypervolumeImprovement
from trieste.acquisition.function import ExpectedImprovement
from trieste.acquisition.function import PredictiveVariance

#Fit the model
model = GaussianProcessRegression(gp_model, num_kernel_samples=10)

# Define acquisition functions. We use the ei rule for exploitation and pv rule for exploration

ei = ExpectedImprovement(constrained_search_space)
rule_ei = EfficientGlobalOptimization(builder=ei)

pv = PredictiveVariance()
rule_pv = EfficientGlobalOptimization(builder=pv)

# Bayesian optimizer
bo = BayesianOptimizer(observer, constrained_search_space)


In [20]:
# Run the Bayesian optimizer for single objective

batch_size = 10 # This number is user defined and determines the number of recommendation made by the BO. Typically 5-10 generations yield good results. For the project, we used a batch of 30. 

# Alternate between rule_ei or rule_pv as parmaters when runnig the BO for exploitation or exploration campaigns, respectively.

bo_result = bo.optimize(batch_size, initial_data(X, Y), model, rule_pv, track_state = False, fit_initial_model=False)

C:\Users\msuvarna\Anaconda3\envs\HAS\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Optimization completed without errors


In [21]:
# Get results from the Bayesian optimizer

bo_initial_data = bo_result.try_get_final_dataset()
bo_X = bo_result.try_get_final_dataset().query_points.numpy()[-batch_size:,:]

bo_X[:,4:] = mm.inverse_transform(bo_X[:,4:]) 

bo_Y = bo_result.try_get_final_dataset().observations.numpy()[-batch_size:,:]
np.set_printoptions(precision=3, suppress=True)

result=(np.concatenate((bo_X, scaley_inv(bo_Y)), axis=1))


In [23]:
# Create dataframe with results 

dfresult = pd.DataFrame(result, columns = ['Zr','Cu','Co','Fe','T','H2:CO','GHSV','STY','YHA'])
dfresult = dfresult.round(2)
dfresult

,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,YHA
0,0.1,0.05,0.11,0.74,268.61,4.0,39253.61,434.79,0.06
1,0.1,0.05,0.10,0.75,269.16,4.0,37422.39,428.54,0.07
2,0.1,0.05,0.12,0.73,268.57,4.0,37007.05,428.01,0.07
3,0.1,0.05,0.12,0.73,268.39,4.0,37250.81,429.13,0.07


In [ ]:
#The above dataframe is the output of the current campiagn recommending catalyst composition worth investigation and its predicted yield.
#In the study we performed these experimental recommendation and measured the actual yield